## Load and setup simple test bench

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../evaluate/")
from aligners.smith_waterman import calculate_smith_waterman_distance
from aligners.bwamem2 import bwa_mem2_align
from aligners.minimap2 import minimap2_align
from aligners.bowtie2 import bowtie2_align

import matplotlib.pyplot as plt

/mnt/SSD2/pholur/environments/dna2vec/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [4]:
string2 = "ATGCTCGATCGATCGATCGAAATCGC"
string1 = "ATCGCCCTATGCTCGATCGATCGATCGAAATCGCAGCTCCTCTGACTCAAGAAGACTCGAATGCTCGATCGATCGATCGAAATCGCAGCCTCGAAGCCTCTTGAAA"
intervals = [1,10,50,100,500,1000,2000,5000,10000,100000]
reference_path = "/home/pholur/dna2vec/evaluate/data/chromosome_2/NC_000002.fasta"
with open(reference_path, "r") as f:
    control = f.read().replace("\n","")
sample_read = "AAATATAAGTCAGATATATCAAAATATAAGTCAGAATTTTACAAATATTGAAGTGTCATATCACATCAGAGTAACAACACCACCTAAGTACCAAATGATGATAATGAAACTTAGACCTACTGGAATTGAGTAGAGGTGAACATCATGTGA"

## Naive Smith-Waterman Distance

In [14]:
import time
start = time.time()
# EXPENSIVE!
print(calculate_smith_waterman_distance(string1, 
                                    string2, 
                                    match_score=2,
                                    mismatch_penalty=-1,
                                    open_gap_penalty=-0.5,
                                    continue_gap_penalty=-0.1,
                                    debug = True))
print(time.time() - start)

Best match:
target            8 ATGCTCGATCGATCGATCGAAATCGC 34
                  0 |||||||||||||||||||||||||| 26
query             0 ATGCTCGATCGATCGATCGAAATCGC 26

{'elapsed time': 0.0021452903747558594, 'distance': -52.0, 'begins': [8]}
0.002481698989868164


## BWA-Mem 2

In [25]:
time_per_read_bwa = []
for i in range(1):
    start = time.time()
    indices = bwa_mem2_align("/mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa", 
                             "/mnt/SSD2/pholur/dna2vec/simulated_reads/reads_500_250_IR1-dot-0_DR1-dot-0_Q10-20_MSv3.fq", 
                             "/home/pholur/dna2vec/evaluate/aligners", "./test.sam");

Looking to launch executable "/home/pholur/dna2vec/evaluate/aligners/bwa-mem2-2.2.1_x64-linux/bwa-mem2.avx2", simd = .avx2
Launching executable "/home/pholur/dna2vec/evaluate/aligners/bwa-mem2-2.2.1_x64-linux/bwa-mem2.avx2"
-----------------------------
Executing in AVX2 mode!!
-----------------------------
* SA compression enabled with xfactor: 8
* Ref file: /mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa
* Entering FMI_search
* Index file found. Loading index from /mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa.bwt.2bit.64
* Reference seq len for bi-index = 6234584141
* sentinel-index: 2011801443
* Count:
0,	1
1,	1846872793
2,	3117292071
3,	4387711349
4,	6234584141

* Reading other elements of the index from files /mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa
* Index prefix: /mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa
* Read 0 ALT contigs
* Done reading Index!!
* Reading reference genome..
* Binary seq file = /mnt/SSD2/pholur/General_Models/data/all/chm1

BWA-MEM2 alignment completed successfully.


No. of OMP threads: 1
Processor is running @2000.219040 MHz
Runtime profile:

	Time taken for main_mem function: 27.14 sec

	IO times (sec) :
	Reading IO time (reads) avg: 0.02, (0.02, 0.02)
	Writing IO time (SAM) avg: 0.05, (0.05, 0.05)
	Reading IO time (Reference Genome) avg: 3.07, (3.07, 3.07)
	Index read time avg: 5.21, (5.21, 5.21)

	Overall time (sec) (Excluding Index reading time):
	PROCESS() (Total compute time + (read + SAM) IO time) : 17.92
	MEM_PROCESS_SEQ() (Total compute time (Kernel + SAM)), avg: 17.84, (17.84, 17.84)

	 SAM Processing time (sec):
	--WORKER_SAM avg: 0.65, (0.65, 0.65)

	Kernels' compute time (sec):
	Total kernel (smem+sal+bsw) time avg: 17.20, (17.20, 17.20)
		SMEM compute avg: 2.32, (2.32, 2.32)
		SAL compute avg: 0.28, (0.28, 0.28)
				MEM_SA avg: 0.16, (0.16, 0.16)

		BSW time, avg: 14.19, (14.19, 14.19)

Important parameter settings: 
	BATCH_SIZE: 512
	MAX_SEQ_LEN_REF: 256
	MAX_SEQ_LEN_QER: 128
	MAX_SEQ_LEN8: 128
	SEEDS_PER_READ: 500
	SIMD_WIDTH8 X: 3

## Minimap2

In [22]:
time_per_read_minimap = []
for i in range(1):
    start = time.time()
    indices = minimap2_align("/mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa", 
                             "/mnt/SSD2/pholur/dna2vec/simulated_reads/reads_500_250_IR1-dot-0_DR1-dot-0_Q10-20_MSv3.fq", #reads_25_250_IR0-dot-0001_DR0-dot-0001_Q60-90_MSv3.fq", 
                             "/home/pholur/dna2vec/evaluate/aligners", 
                             "./test.sam");

[M::mm_idx_gen::62.812*1.49] collected minimizers
[M::mm_idx_gen::77.606*1.77] sorted minimizers
[M::main::77.606*1.77] loaded/built the index for 25 target sequence(s)
[M::mm_mapopt_update::77.606*1.77] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 25
[M::mm_idx_stat::80.496*1.75] distinct minimizers: 382283879 (95.25% are singletons); average occurrences: 1.364; average spacing: 5.978; total length: 3117292070
[M::worker_pipeline::81.751*1.76] mapped 12500 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: /home/pholur/dna2vec/evaluate/aligners/minimap2/minimap2 -ax sr -B 5 -O 30 -E 30 /mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa /mnt/SSD2/pholur/dna2vec/simulated_reads/reads_500_250_IR1-dot-0_DR1-dot-0_Q10-20_MSv3.fq
[M::main] Real time: 81.901 sec; CPU: 144.249 sec; Peak RSS: 11.568 GB


minimap2 alignment completed successfully.


## Bowtie2

In [4]:
time_per_read_bowtie = []
reads = []
with open("/mnt/SSD5/pholur/dna/reads.txt", "r") as f:
    reads = f.readlines()

reads = [read.strip()[:] for read in reads if read.strip() != ""]
# reads = ["".join(["A"]*250)]*250
import time
for i in range(1):
    start = time.time()
    # indices = bowtie2_align("/home/pholur/dna2vec/evaluate/data/chromosome_2/NC_000002", [sample_read]*i, "/home/pholur/dna2vec/evaluate/aligners/bowtie2-2.5.1-linux-x86_64", "./test.sam");
    indices = bowtie2_align("/mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa", 
                            reads,
                            # "/mnt/SSD2/pholur/dna2vec/External_Data/sample_250_illumina_wgs.fastq",
                            # "/mnt/SSD2/pholur/dna2vec/simulated_reads/reads_25_250_IR0-dot-0001_DR0-dot-0001_Q60-90_MSv3.fq", 
                            # "/mnt/SSD2/pholur/dna2vec/simulated_reads/reads_500_250_IR1-dot-0_DR1-dot-0_Q10-20_MSv3.fq",
                            "/home/pholur/dna2vec/evaluate/aligners/bowtie2-2.5.1-linux-x86_64", 
                            "./test.sam");

Bowtie2 alignment completed successfully.
Bowtie2 alignment completed successfully.


10000 reads; of these:
  10000 (100.00%) were unpaired; of these:
    28 (0.28%) aligned 0 times
    9246 (92.46%) aligned exactly 1 time
    726 (7.26%) aligned >1 times
99.72% overall alignment rate


In [4]:
from aligners.helpers import load_reference_genome, compute_sw
reference = load_reference_genome("/mnt/SSD2/pholur/General_Models/data/all/chm13v2.0.fa")

In [5]:
compute_sw("./test.sam", reference)

  4%|▍         | 424/10027 [00:05<02:17, 69.70it/s]

250
GATAATAAATCATTTGATCACACTATTAATGGGATATTCGAATATTTCAAAGAATTCAGCACAGTTGACAGGTCTAGGCATATAAGACTCTCAGGCAGCCTGAAGATTAGCCATGAGGTGAGCCAGGTGGTGGCAGCTTCACCCTGTCGATGCCAGAACTCATTCTGACATCAGGACTGGGCAGACGGGCTTCCTCGGGCTGCTTGTTCCATGAGAATTCTCCTGGAATGATGTGCTTGGATGAAAAAGA
AATGAGTAGTAACATTGAATTAGCACTCAGAATCTTCCTGCCATGAAGTCTGTTGTGTTCTCTACATTTTCTGACACCCGCTTTTTGTGTAAAGTTTTGTTTATTACCTGATGAATAGAACAAAACTTGTTCGTGCCCATTAAACAGGTAATTCATGTTTTCGAAAGATAAGCATTATCTCTCCTTTATCATTCCCAATAATTCAACTCTGGAGTCCTGATCAGTGTGGCTTACAAACTGGAAGACAAAAACATTTCGAAAAAACCTAATAGCTAAGTTAGTTTTttaagctattattattgttaaaataataatagtgaactattatttttagttcacttgaaaaaaatcacttggGCATTGTAGAGACACAGCACTGTGCCAACAAAAAGCTGATTACTAAAGGtaggatgaccatatatcctggtttgtccagaaaaaatgtctgagtgtaattatcagtaggaccctctttcagtctcagaagtctccagatttggataataaatcatttgatcacactaTTAATGggatatttgaatatttcaaagAATTCAGCACAGTTGACAGGTCTAGGCATATAAGACTCTCAGGCAGCCTGAAGATTAGCCATGAGGTGAGCCAGGTGGTGGCAGCTTCACCCTGTCGATGCCAGAACTCATTCTGACATCAGGACTGGGCAGACGGGCTTCCTCAGGCTGCTTGTTCCATGAGAATTCTCCTGGAATGATGTGCTTGGATGAAA

 13%|█▎        | 1323/10027 [00:18<02:04, 70.16it/s]